## 복합 키워드

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()


def extract_dynamic_keywords(text, category):
    """
    문장에서 복합 키워드를 순차적으로 결합하며 추출.
    """
    # 형태소 분석
    pos_tags = mecab.pos(text)
    extracted_keywords = [word for word, tag in pos_tags if tag in ["NNG", "NNP"]]

    # 카테고리 키워드 집합 생성
    category_keywords = set(
        categories[category]["locations"] +
        categories[category]["smell_types"] +
        categories[category]["intensity_adjectives"]
    )

    # 복합 키워드 처리
    final_keywords = []
    temp_combination = []

    for i, word in enumerate(extracted_keywords):
        temp_combination.append(word)  # 단어를 임시 결합 리스트에 추가
        combined = " ".join(temp_combination)  # 결합된 문자열 생성

        if combined in category_keywords:  # 결합 결과가 카테고리 키워드에 있으면
            final_keywords.append(combined)  # 키워드로 추가
            temp_combination = []  # 임시 결합 초기화
        elif i == len(extracted_keywords) - 1:  # 마지막 단어 처리
            # 마지막 단어까지 결합이 없으면 개별 단어 추가
            final_keywords.extend(temp_combination)

    # 문장 길이에 따라 동적 키워드 수 결정
    total_words = len(pos_tags)
    num_keywords = max(1, int(total_words * 0.5))

    return final_keywords[:num_keywords]  # 키워드 제한 적용


In [10]:
categories = {
    "odor": {
        "locations": ["방음벽 공사 현장", "공장"],
        "smell_types": ["화학물 냄새", "음식물 쓰레기 냄새"],
        "intensity_adjectives": ["심각하게", "조용히"],
        "effects": ["건강에 악영향을 미칩니다.", "불쾌감을 유발합니다."]
    }
}


text = "방음벽 공사 현장에서 화학물 냄새가 조용히 발생하고 있습니다."

# 테스트 실행
keywords = extract_dynamic_keywords(text, "odor")
print(keywords)


['방음벽 공사 현장', '화학', '물', '냄새', '발생']


## 띄어쓰기

In [19]:
def extract_keywords_by_space(text, category):
    """
    띄어쓰기를 기준으로 복합 키워드를 추출하며 조사를 제거.
    """
    import re

    # 문장을 띄어쓰기 기준으로 분리
    words = text.split()

    # 조사 목록 (필요에 따라 추가)
    particles = ["에서", "으로", "에", "의", "을", "를", "이", "가", "과", "와", "도", "는", "다", "만"]

    # 단어에서 조사를 제거
    def remove_particle(word):
        for particle in particles:
            if word.endswith(particle):
                return re.sub(f"{particle}$", "", word)  # 조사 제거
        return word

    # 카테고리 키워드 목록 (집합으로 변환)
    category_keywords = set(
        categories[category]["locations"] +
        categories[category]["smell_types"] +
        categories[category]["intensity_adjectives"]
    )

    # 복합 키워드 처리
    final_keywords = []
    temp_combination = []

    i = 0
    while i < len(words):
        clean_word = remove_particle(words[i])  # 단어에서 조사 제거
        temp_combination.append(clean_word)  # 단어 추가
        combined = " ".join(temp_combination)  # 결합된 문자열 생성

        if combined in category_keywords:  # 결합 결과가 정확히 일치하면
            final_keywords.append(combined)  # 키워드로 저장
            temp_combination = []  # 임시 결합 초기화
        elif i == len(words) - 1:  # 마지막 단어 처리
            # temp_combination이 키워드와 매칭되지 않으면 버림
            temp_combination = []  # 초기화
        i += 1  # 다음 단어로 이동

    return final_keywords


categories = {
    "odor": {
        "locations": ["방음벽 공사 현장", "공장"],
        "smell_types": ["화학물 냄새", "음식물 쓰레기 냄새"],
        "intensity_adjectives": ["심각하게", "조용히"],
        "effects": ["건강에 악영향을 미칩니다.", "불쾌감을 유발합니다."]
    }
}

text = "방음벽 공사 현장에서 화학물 냄새가 조용히 발생하고 있습니다."

# 테스트 실행
keywords = extract_keywords_by_space(text, "odor")
print(keywords)


['방음벽 공사 현장', '화학물 냄새', '조용히']


## n-gram

### mecab

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()

print(mecab.pos('쓰레기 매립장에서 악취가 쾌쾌하게 발생하고 있습니다. 이로 인해 주민들의 일상생활에 심각한 지장을 초래하고 있으며, 특히 아이들과 노약자들의 건강이 우려됩니다'))

[('쓰레기', 'NNG'), ('매립장', 'NNG'), ('에서', 'JKB'), ('악취', 'NNG'), ('가', 'JKS'), ('쾌쾌하', 'VA'), ('게', 'EC'), ('발생', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF'), ('이', 'NP'), ('로', 'JKB'), ('인해', 'VV+EC'), ('주민', 'NNG'), ('들', 'XSN'), ('의', 'JKG'), ('일상', 'NNG'), ('생활', 'NNG'), ('에', 'JKB'), ('심각', 'XR'), ('한', 'XSA+ETM'), ('지장', 'NNG'), ('을', 'JKO'), ('초래', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('으며', 'EC'), (',', 'SC'), ('특히', 'MAG'), ('아이', 'NNG'), ('들', 'XSN'), ('과', 'JC'), ('노약자', 'NNG'), ('들', 'XSN'), ('의', 'JKG'), ('건강', 'NNG'), ('이', 'JKS'), ('우려', 'NNG'), ('됩니다', 'XSA+EC')]


In [27]:
from konlpy.tag import Mecab

mecab = Mecab()

def extract_keywords_with_ngram(text, category):
    """
    문장에서 N-gram 방식으로 키워드를 추출.
    """
    # 형태소 분석
    pos_tags = mecab.pos(text)

    # 필요한 품사 추출 및 결합
    words = []
    temp_word = ""
    skip_next = False

    for i, (word, tag) in enumerate(pos_tags):
        if skip_next:  # 이미 결합 처리한 경우
            skip_next = False
            continue

        # 'XR' + 'XSA' 패턴 결합 처리
        if tag == "XR" and i + 1 < len(pos_tags) and pos_tags[i + 1][1] == "XSA":
            temp_word = word + pos_tags[i + 1][0]  # 어근 + 접사 결합
            if i + 2 < len(pos_tags) and pos_tags[i + 2][1] == "EC":  # 연결 어미까지 포함
                temp_word += pos_tags[i + 2][0]
                skip_next = True  # 이미 처리한 어미 건너뛰기
            words.append(temp_word)
            skip_next = True  # 접사 건너뛰기
            temp_word = ""
        elif tag in ["NNG", "NNP", "MAG", "VA"]:  # 명사, 부사, 형용사 추가
            words.append(word)

    # 카테고리 키워드 집합
    category_keywords = set(
        categories[category]["locations"] +
        categories[category]["smell_types"] +
        categories[category]["intensity_adjectives"]
    )

    # N-gram으로 키워드 매칭
    final_keywords = []
    for n in range(1, len(words) + 1):  # 1-gram부터 전체 길이까지
        for i in range(len(words) - n + 1):
            ngram = " ".join(words[i:i + n])  # N-gram 생성
            if ngram in category_keywords:  # 키워드 매칭
                final_keywords.append(ngram)

    # 중복 제거 및 순서 유지
    final_keywords = list(dict.fromkeys(final_keywords))

    return final_keywords

# 테스트 데이터
categories = {
    "odor": {
        "locations": ["아파트 공사장", "하수구"],
        "smell_types": ["비린내", "하수구 냄새"],
        "intensity_adjectives": ["쾌쾌하게", "축축하게"]
    }
}

text = "아파트 공사장에서 쾌쾌하게 비린내가 발생하고 있습니다."

# 실행
keywords = extract_keywords_with_ngram(text, "odor")
print(keywords)

['쾌쾌하게', '비린내', '아파트 공사장']


### komoran

In [37]:
from konlpy.tag import Komoran
komoran = Komoran()

print(komoran.pos('쓰레기 매립장에서 쾌쾌하게 악취가 발생하고 있습니다.'))
print()
print(komoran.pos('쓰레기 매립장에서 악취가 쾌쾌하게 발생하고 있습니다. 이로 인해 주민들의 일상생활에 심각한 지장을 초래하고 있으며, 특히 아이들과 노약자들의 건강이 우려됩니다'))
print()
print(komoran.pos('쓰레기 매립장에서 지독하게 악취가 발생하고 있습니다.'))

[('쓰레기', 'NNP'), ('매립장', 'NNG'), ('에서', 'JKB'), ('쾌', 'NNG'), ('쾌', 'NNG'), ('하', 'XSV'), ('게', 'EC'), ('악취', 'NNP'), ('가', 'JKS'), ('발생', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF')]

[('쓰레기', 'NNP'), ('매립장', 'NNG'), ('에서', 'JKB'), ('악취', 'NNP'), ('가', 'JKS'), ('쾌', 'NNG'), ('쾌', 'NNG'), ('하', 'XSV'), ('게', 'EC'), ('발생', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF'), ('이', 'NP'), ('로', 'JKB'), ('인해', 'NNP'), ('주민', 'NNG'), ('들', 'XSN'), ('의', 'JKG'), ('일상생활', 'NNG'), ('에', 'JKB'), ('심각', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('지장', 'NNG'), ('을', 'JKO'), ('초래', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VV'), ('으며', 'EC'), (',', 'SP'), ('특히', 'MAG'), ('아이들', 'NNP'), ('과', 'JC'), ('노약자', 'NNG'), ('들', 'XSN'), ('의', 'JKG'), ('건강', 'NNG'), ('이', 'JKS'), ('우려', 'NNG'), ('되', 'XSV'), ('ㅂ니다', 'EC')]

[('쓰레기', 'NNP'), ('매립장', 'NNG'), ('에서', 'JKB'), ('지독', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('악취', 'NNP'), ('가', 'JKS'), ('발생', 'NNG'), ('하',

In [39]:
text_lst = ['쓰레기 매립장에서 고약한 냄새가 자욱하게 발생하고 있습니다. 이 문제로 인해 주민들이 집을 떠나 이사를 고민하고 있습니다',
            '쓰레기 매립장에서 고약한 냄새가 자욱하게 발생하고 있습니다. 악취로 인해 외부에서 온 방문객들이 심각한 불쾌감을 느끼고 있습니다',
            '쓰레기 매립장에서 고약한 냄새가 자욱하게 발생하고 있습니다. 지역 행사와 축제에도 부정적인 영향을 미치고 있습니다',
            '쓰레기 매립장에서 고약한 냄새가 자욱하게 발생하고 있습니다. 이 문제는 더 이상 방치할 수 없는 심각한 상태에 이르렀습니다']

for text in text_lst:
    print(komoran.pos(text), '\n')

[('쓰레기', 'NNP'), ('매립장', 'NNG'), ('에서', 'JKB'), ('고약', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('냄새', 'NNG'), ('가', 'JKS'), ('자욱', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('발생', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF'), ('이', 'MM'), ('문제', 'NNG'), ('로', 'JKB'), ('인해', 'NNP'), ('주민', 'NNG'), ('들', 'XSN'), ('이', 'JKS'), ('집', 'NNG'), ('을', 'JKO'), ('떠나', 'VV'), ('아', 'EC'), ('이사', 'NNG'), ('를', 'JKO'), ('고민', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VV'), ('습니다', 'EC')] 

[('쓰레기', 'NNP'), ('매립장', 'NNG'), ('에서', 'JKB'), ('고약', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('냄새', 'NNG'), ('가', 'JKS'), ('자욱', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('발생', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF'), ('악취', 'NNP'), ('로', 'JKB'), ('인해', 'NNP'), ('외부', 'NNG'), ('에서', 'JKB'), ('오', 'VV'), ('ㄴ', 'ETM'), ('방문객', 'NNG'), ('들', 'XSN'), ('이', 'JKS'), ('심각', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('불쾌감', 'NNG'), ('을', 'JKO'), ('느끼', 'VV'), ('고', 'EC'), ('있', 'VV'), (

In [45]:
from konlpy.tag import Komoran
import re

class AdvancedKoreanKeywordExtractor:
    def __init__(self, categories):
        self.komoran = Komoran()
        self.categories = categories
        
        # 기본 키워드 확장 패턴
        self.keyword_expansion_patterns = {
            # 강도/감정 표현 확장
            'intensity_adjectives': [
                r'^(고약|자욱|엄청|매우|너무|정말|몹시|극도로|특히).*',
                r'.*(?:하다|스럽다|롭다|덥다)$',
                r'.*(?:게|히)$'
            ],
            # 문제 상황 키워드 확장
            'problem_keywords': [
                r'.*(?:문제|이사|고민|어려움|불편).*',
                r'(?:떠나|포기|힘들|불가능).*'
            ]
        }
        
    def _normalize_word(self, word, tag):
        """
        단어 정규화 및 확장 로직
        """
        # 형용사, 부사 관련 정규화
        if tag in ['VA', 'MAG', 'XSV', 'XSA', 'XR']:
            word = re.sub(r'(하|스럽|롭|덥)다?$', '', word)
            word = re.sub(r'(게|히)$', '', word)
        
        return word

    def _match_expanded_patterns(self, word, pattern_type):
        """
        확장 패턴 매칭
        """
        for pattern in self.keyword_expansion_patterns.get(pattern_type, []):
            if re.match(pattern, word):
                return True
        return False

    def extract_keywords(self, text, category):
        """
        고급 키워드 추출 메서드
        """
        # 형태소 분석
        pos_tags = self.komoran.pos(text)
        
        # 카테고리 키워드 집합
        category_keywords = {
            key: set(values) 
            for key, values in self.categories[category].items()
        }
        
        # 추출 대상 품사 확장
        target_tags = ["NNG", "NNP", "VA", "MAG", "XSV", "XSA", "XR", "MM"]
        
        # 키워드 추출 결과 저장
        final_keywords = set()
        
        # 1. 직접 키워드 매칭
        for word, tag in pos_tags:
            if tag in target_tags:
                normalized_word = self._normalize_word(word, tag)
                
                # 각 카테고리별 키워드 매칭
                for category_type, keywords in category_keywords.items():
                    if normalized_word in keywords:
                        final_keywords.add(normalized_word)
                    
                    # 확장 패턴 매칭
                    if category_type == 'intensity_adjectives':
                        if self._match_expanded_patterns(word, 'intensity_adjectives'):
                            final_keywords.add(normalized_word)
        
        # 2. N-gram 키워드 추출
        words = [word for word, tag in pos_tags if tag in target_tags]
        for n in range(2, min(4, len(words) + 1)):
            for i in range(len(words) - n + 1):
                ngram = " ".join(words[i:i + n])
                for keywords in category_keywords.values():
                    if ngram in keywords:
                        final_keywords.add(ngram)
        
        # 3. 문제 상황 키워드 추출
        problem_keywords = []
        for word, tag in pos_tags:
            if self._match_expanded_patterns(word, 'problem_keywords'):
                problem_keywords.append(word)
        
        if problem_keywords:
            final_keywords.update(problem_keywords)
        
        # 4. 부분 일치 및 컨텍스트 기반 키워드
        for keyword_list in category_keywords.values():
            for keyword in keyword_list:
                if keyword in text:
                    final_keywords.add(keyword)
        
        return list(final_keywords)

# 카테고리 확장
categories = {
    "odor": {
        "locations": ["쓰레기 매립장", "매립장", "쓰레기장", "방음벽 공사 현장"],
        "smell_types": [
            "악취", "비린내", "썩은내", "고약한내", 
            "지독한냄새", "역겨운냄새", '화학물 냄새', '매캐한'
        ],
        "intensity_adjectives": [
            "쾌쾌", "쾌쾌하다", "심각", "지독", 
            "지독하다", "심각하다", "역겹다", 
            "강렬하다", "고약하다", "자욱하다", "조용하게", '강압적으로'
        ]
    }
}

# 사용 예시
extractor = AdvancedKoreanKeywordExtractor(categories)

# 테스트 문장들
test_sentences = [
    "쓰레기 매립장에서 지독하게 악취가 발생하고 있습니다.",
    "고약한 냄새로 인해 주민들이 집을 떠나 이사를 고민하고 있습니다.",
    "매립장의 자욱한 비린내가 인근 지역에 심각한 문제를 일으키고 있습니다.",
    '방음벽 공사 현장에서 연소된 화학물 냄새가 조용히 스며들게 발생하고 있습니다. 이 문제는 지역 전체의 건강과 안전을 위협하고 있습니다',
    '쓰레기 매립장에서 매캐한 냄새가 강압적으로 발생하고 있습니다. 이 문제로 인해 주변 부동산 가격이 하락하고 있습니다'
]

# 각 문장에 대해 키워드 추출
for sentence in test_sentences:
    print(f"문장: {sentence}")
    keywords = extractor.extract_keywords(sentence, "odor")
    print(f"키워드: {keywords}\n")

문장: 쓰레기 매립장에서 지독하게 악취가 발생하고 있습니다.
키워드: ['악취', '지독', '매립장', '쓰레기 매립장']

문장: 고약한 냄새로 인해 주민들이 집을 떠나 이사를 고민하고 있습니다.
키워드: ['고약', '떠나', '고민', '이사']

문장: 매립장의 자욱한 비린내가 인근 지역에 심각한 문제를 일으키고 있습니다.
키워드: ['문제', '비린내', '자욱', '심각', '매립장']

문장: 방음벽 공사 현장에서 연소된 화학물 냄새가 조용히 스며들게 발생하고 있습니다. 이 문제는 지역 전체의 건강과 안전을 위협하고 있습니다
키워드: ['문제', '화학물 냄새', '조용', '방음벽 공사 현장']

문장: 쓰레기 매립장에서 매캐한 냄새가 강압적으로 발생하고 있습니다. 이 문제로 인해 주변 부동산 가격이 하락하고 있습니다
키워드: ['강압적으로', '문제', '매캐한', '쓰레기 매립장', '매립장']



## 머신러닝 & 임베딩

In [19]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate, GlobalMaxPooling1D
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Komoran
import re
import json
import faiss

class SemanticKeywordExtractor:
    def __init__(self, categories, embedding_dim=100, max_words=1000):
        self.komoran = Komoran()
        self.categories = categories
        self.embedding_dim = embedding_dim
        self.max_words = max_words
            
        # 한국어 Word2Vec 사전 학습 모델 (예시 경로, 실제 경로로 변경 필요)
        self.word2vec_model = self._load_word2vec_model()
        
        # 벡터 크기 확인 (디버깅용)
        # if self.word2vec_model:
        #     for word in self.word2vec_model.index_to_key:
        #         print(f"Word: {word}, Vector Shape: {self.word2vec_model[word].shape}")
        
        # 카테고리 벡터 생성
        self.category_vectors = self._create_category_vectors()
        
        # Faiss 인덱스 생성 (고속 유사도 검색)
        self.faiss_index = self._build_faiss_index()
        
        # 딥러닝 키워드 분류기 모델
        self.keyword_classifier = self._build_keyword_classifier()

    def _load_word2vec_model(self):
        """
        Word2Vec 또는 FastText 모델 로드 함수
        """
        try:
            path = '/home/yjtech2/Desktop/yurim/LLM/Pre_processing/smell_keyword/kor_w2v'
            model = KeyedVectors.load_word2vec_format(path, binary=False)
            print("Word2Vec 모델 로드 성공!")
            return model
        except Exception as e:
            print(f"모델 로드 실패: {e}")
            return None

    def _create_category_vectors(self):
        category_vectors = {}
        for category, keywords in self.categories.items():
            category_vector = []
            for keyword_list in keywords.values():
                for keyword in keyword_list:
                    # 단어 벡터를 가져오거나 없는 경우 0 벡터 사용
                    keyword_vectors = [
                        self.word2vec_model[word] if word in self.word2vec_model else np.zeros(self.embedding_dim)
                        for word in keyword.split()
                    ]
                    
                    # keyword_vectors가 비어있지 않고, 벡터 크기가 모두 동일한 경우 처리
                    if keyword_vectors and all(vec.shape == (self.embedding_dim,) for vec in keyword_vectors):
                        category_vector.append(np.mean(keyword_vectors, axis=0))

            # category_vector가 비어있지 않으면 평균을 저장, 그렇지 않으면 0 벡터 저장
            if category_vector:
                category_vectors[category] = np.mean(category_vector, axis=0)
            else:
                category_vectors[category] = np.zeros(self.embedding_dim)
        return category_vectors



    def _build_faiss_index(self):
        """
        Faiss 인덱스 생성 (고속 유사도 검색)
        """
        # 카테고리 벡터로 Faiss 인덱스 생성
        category_matrix = np.array(list(self.category_vectors.values()))
        dimension = category_matrix.shape[1]
        
        # L2 거리 기반 인덱스 생성
        index = faiss.IndexFlatL2(dimension)
        index.add(category_matrix)
        
        return index

    def _build_keyword_classifier(self):
        """
        딥러닝 키워드 분류기 모델 구축
        """
        # 입력 레이어
        input_layer = Input(shape=(None,))
        
        # 임베딩 레이어
        embedding_layer = Embedding(
            input_dim=self.max_words, 
            output_dim=self.embedding_dim, 
            mask_zero=True
        )(input_layer)
        
        # LSTM 레이어
        lstm_layer = LSTM(64, return_sequences=True)(embedding_layer)
        
        # 글로벌 맥스 풀링
        pooling_layer = GlobalMaxPooling1D()(lstm_layer)
        
        # 다중 레이블 분류 출력 레이어
        output_layer = Dense(
            len(self.categories), 
            activation='sigmoid'
        )(pooling_layer)
        
        model = Model(inputs=input_layer, outputs=output_layer)
        model.compile(
            optimizer='adam', 
            loss='binary_crossentropy', 
            metrics=['accuracy']
        )
        
        return model

    def preprocess_text(self, text):
        """
        텍스트 전처리
        """
        # Komoran 형태소 분석
        pos_tags = self.komoran.pos(text)
        print(f"형태소 분석 결과: {pos_tags}")  # 디버깅용 출력

        # 명사, 형용사, 부사 추출
        keywords = [
            word for word, tag in pos_tags 
            if tag in ['NNG', 'NNP', 'VA', 'MAG', 'XSA']
        ]
        return keywords


    def semantic_similarity(self, text_keywords):
        """
        의미론적 유사성 분석
        """
        # 키워드 벡터 변환
        keyword_vectors = [
            self.word2vec_model[keyword] if keyword in self.word2vec_model else np.zeros(self.embedding_dim)
            for keyword in text_keywords
        ]
        
        # 카테고리와의 유사도 계산
        similarities = {}
        for category, category_vector in self.category_vectors.items():
            # 키워드 벡터와 카테고리 벡터 간 코사인 유사도 계산
            category_similarities = [
                cosine_similarity([vec], [category_vector])[0][0]
                for vec in keyword_vectors
            ]
            similarities[category] = np.mean(category_similarities)
        
        return similarities

    def extract_keywords(self, text, top_k=10):
        """
        키워드 추출 메인 메서드 (형태 유사성 기반)
        """
        # 텍스트 전처리: 문장에서 명사, 형용사, 부사를 추출
        keywords = self.preprocess_text(text)
        
        # 카테고리별 키워드 필터링 (형태 유사성 포함)
        matched_keywords = []
        for category, keyword_types in self.categories.items():
            for keyword_list in keyword_types.values():
                for kw in keyword_list:
                    # 문장 키워드 중에서 부분 문자열 포함 또는 접두/접미가 유사한 경우 추출
                    for word in keywords:
                        if kw in word or word in kw:
                            matched_keywords.append(kw)
        
        # 중복 제거 및 상위 K개 키워드 반환
        matched_keywords = list(dict.fromkeys(matched_keywords))  # 중복 제거
        return matched_keywords[:top_k]  # 상위 K개 키워드 반환




# 카테고리 정의
categories = {
    "odor": {
        "locations": ["쓰레기 매립장", "매립장", "쓰레기장"],
        "smell_types": [
            "악취", "비린내", "썩은내", "고약한내", 
            "지독한냄새", "역겨운냄새"
        ],
        "intensity_adjectives": [
            "쾌쾌", "심각", "지독하게", 
            "심각하다", "역겹다", 
            "강렬하다", "고약하다", '자욱한'
        ]
    }
}

# 키워드 추출기 초기화
extractor = SemanticKeywordExtractor(categories, embedding_dim=150)

# 테스트 문장들
test_sentences = [
    "쓰레기 매립장에서 지독하게 악취가 발생하고 있습니다.",
    "고약한 냄새로 인해 주민들이 집을 떠나 이사를 고민하고 있습니다.",
    "매립장의 자욱한 비린내가 인근 지역에 심각한 문제를 일으키고 있습니다."
]

# 각 문장에 대해 키워드 추출
for sentence in test_sentences:
    print(f"문장: {sentence}")
    keywords = extractor.extract_keywords(sentence)
    print(f"키워드: {keywords}\n")

Word2Vec 모델 로드 성공!
문장: 쓰레기 매립장에서 지독하게 악취가 발생하고 있습니다.
형태소 분석 결과: [('쓰레기', 'NNP'), ('매립장', 'NNG'), ('에서', 'JKB'), ('지독', 'XR'), ('하', 'XSA'), ('게', 'EC'), ('악취', 'NNP'), ('가', 'JKS'), ('발생', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF')]
키워드: ['쓰레기 매립장', '매립장', '쓰레기장', '악취', '지독하게', '심각하다', '강렬하다', '고약하다']

문장: 고약한 냄새로 인해 주민들이 집을 떠나 이사를 고민하고 있습니다.
형태소 분석 결과: [('고약', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('냄새', 'NNG'), ('로', 'JKB'), ('인해', 'NNP'), ('주민', 'NNG'), ('들', 'XSN'), ('이', 'JKS'), ('집', 'NNG'), ('을', 'JKO'), ('떠나', 'VV'), ('아', 'EC'), ('이사', 'NNG'), ('를', 'JKO'), ('고민', 'NNG'), ('하', 'XSV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF')]
키워드: ['지독한냄새', '역겨운냄새', '지독하게', '심각하다', '강렬하다', '고약하다']

문장: 매립장의 자욱한 비린내가 인근 지역에 심각한 문제를 일으키고 있습니다.
형태소 분석 결과: [('매립장', 'NNG'), ('의', 'JKG'), ('자욱', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('비린내', 'NNG'), ('가', 'JKS'), ('인근', 'NNG'), ('지역', 'NNG'), ('에', 'JKB'), ('심각', 'XR'), ('하', 'XSA'), ('ㄴ', 'ETM'), ('문제', 'NNG'),

In [18]:
from konlpy.tag import Mecab

mecab = Mecab()
print(mecab.pos('매립장의 자욱한 비린내가 인근 지역에 심각한 문제를 일으키고 있습니다.'))

[('매립장', 'NNG'), ('의', 'JKG'), ('자욱', 'XR'), ('한', 'XSA+ETM'), ('비린내', 'NNG'), ('가', 'JKS'), ('인근', 'NNG'), ('지역', 'NNG'), ('에', 'JKB'), ('심각', 'XR'), ('한', 'XSA+ETM'), ('문제', 'NNG'), ('를', 'JKO'), ('일으키', 'VV'), ('고', 'EC'), ('있', 'VX'), ('습니다', 'EF'), ('.', 'SF')]
